In [1]:
%load_ext autoreload
%autoreload 2

import os
import random
import sys

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from pathlib import Path
from hydra import initialize_config_dir, compose
from hydra.utils import instantiate
from omegaconf import OmegaConf
import shutil
import tqdm
import torch
import matplotlib.pyplot as plt

from tbfm import film
from tbfm import multisession
from tbfm import utils

DATA_DIR = "/home/mmattb/Projects/opto-coproc/data"
sys.path.append(DATA_DIR)
# imported from JNE project
import dataset

EMBEDDING_REST_SUBDIR = "embedding_rest"

conf_dir = Path("./conf").resolve()

# Initialize Hydra with the configuration directory
with initialize_config_dir(config_dir=str(conf_dir), version_base=None):
    # Compose the configuration
    cfg = compose(config_name="config")   # i.e. conf/config.yaml

DEVICE = "cuda" #cfg.device
WINDOW_SIZE = cfg.data.trial_len
NUM_HELD_OUT_SESSIONS = 0
session_ids, _ = multisession.gather_session_ids(DATA_DIR, num_held_out_sessions=NUM_HELD_OUT_SESSIONS)
NUM_REPEATS = 3

RESULTS_DIR = "singles"

cfg.training.epochs = 20001

In [3]:
# Dividing the work up so we can run two GPUs...
n = len(session_ids)
top_ids = session_ids[:n//2]
bottom_ids = session_ids[n//2:]
print(top_ids)
print("----")
print(bottom_ids)

# my_ids = top_ids

my_ids = ['MonkeyG_20150914_Session1_S1', 'MonkeyG_20150922_Session2_S1', 'MonkeyJ_20160625_Session4_S1', 'MonkeyJ_20160428_Session2_S1', 'MonkeyJ_20160502_Session1_S1', 'MonkeyJ_20160702_Session2_S1', 'MonkeyG_20150915_Session3_S1', 'MonkeyG_20150922_Session3_S1', 'MonkeyJ_20160630_Session3_S1', 'MonkeyJ_20160630_Session1_S1', 'MonkeyG_20150915_Session2_S1', 'MonkeyG_20150925_Session2_S1', 'MonkeyG_20150917_Session3_M1', 'MonkeyJ_20160429_Session1_S1', 'MonkeyJ_20160428_Session3_S1', 'MonkeyG_20150915_Session4_S1', 'MonkeyG_20150917_Session1_S1', 'MonkeyG_20150917_Session2_S1', 'MonkeyG_20150918_Session1_M1', 'MonkeyJ_20160624_Session3_S1']

['MonkeyG_20150921_Session5_S1', 'MonkeyG_20150922_Session3_S1', 'MonkeyJ_20160702_Session4_S1', 'MonkeyJ_20160627_Session2_S1', 'MonkeyJ_20160630_Session3_S1', 'MonkeyJ_20160625_Session5_S1', 'MonkeyG_20150916_Session4_S1', 'MonkeyG_20150915_Session4_S1', 'MonkeyG_20150917_Session1_S1', 'MonkeyG_20150922_Session1_S1', 'MonkeyJ_20160429_Session3_S1', 'MonkeyG_20150914_Session1_S1', 'MonkeyJ_20160702_Session2_S1', 'MonkeyG_20150917_Session2_M1', 'MonkeyJ_20160627_Session1_S1', 'MonkeyJ_20160502_Session1_S1', 'MonkeyJ_20160426_Session3_S1', 'MonkeyG_20150915_Session3_S1', 'MonkeyG_20150917_Session3_S1', 'MonkeyG_20150918_Session1_S1']
----
['MonkeyJ_20160625_Session4_S1', 'MonkeyJ_20160624_Session4_S1', 'MonkeyJ_20160429_Session1_S1', 'MonkeyG_20150917_Session1_M1', 'MonkeyJ_20160630_Session1_S1', 'MonkeyJ_20160428_Session3_S1', 'MonkeyG_20150917_Session3_M1', 'MonkeyG_20150922_Session2_S1', 'MonkeyG_20150914_Session3_S1', 'MonkeyG_20150915_Session5_S1', 'MonkeyG_20150921_Session3_S1', '

In [4]:
# Rest embeddings
embeddings_rest = multisession.load_rest_embeddings(my_ids, device=DEVICE)

In [ ]:
for session_id in my_ids:
    print("-----------------------------------------", session_id)
    out_dir = os.path.join(RESULTS_DIR, session_id)
    
    try:
        shutil.rmtree(out_dir)
    except OSError:
        pass
    os.makedirs(out_dir, mode=0o777, exist_ok=False)

    _embeddings_rest = {session_id: embeddings_rest[session_id]}
    
    for nidx in range(NUM_REPEATS):
        print("@@@@@", nidx)
        
        d, held_out_session_ids = multisession.load_stim_batched(                                                             
            window_size=WINDOW_SIZE,                                                               
            session_subdir="torchraw",                                                     
            data_dir=DATA_DIR,
            held_in_session_ids=[session_id,],
            batch_size=6000,
            num_held_out_sessions=0,                                                      
        )
        data_train, data_test = d.train_test_split(6000, test_cut=2500)

        ms = multisession.build_from_cfg(cfg, data_train, device=DEVICE, quiet=True)
        model_optims = multisession.get_optims(cfg, ms)


        embeddings_stim, results = multisession.train_from_cfg(
            cfg,
            ms,
            data_train,
            model_optims,
            _embeddings_rest,
            data_test=data_test,
            test_interval=10000,
            epochs=cfg.training.epochs)

        torch.save(embeddings_stim, os.path.join(out_dir, f"es_{nidx}.torch"))
        torch.save(results, os.path.join(out_dir, f"results_{nidx}.torch"))

----------------------------------------- MonkeyG_20150921_Session5_S1
@@@@@ 0
---- 0 0.7828640341758728 1.0461922883987427 0.36576947569847107 0.051312245428562164
---- 10000 0.3620448112487793 0.8020395040512085 0.6393890380859375 0.27419164776802063
---- 20000 0.33237460255622864 0.8536006212234497 0.6737083792686462 0.2272273153066635
Final: 0.853599488735199 0.2272282987833023
@@@@@ 1
---- 0 0.7829082012176514 1.0464801788330078 0.36550837755203247 0.05110074579715729
---- 10000 0.35431382060050964 0.7978336811065674 0.6478596925735474 0.27811527252197266
---- 20000 0.3299413323402405 0.860785186290741 0.6759799718856812 0.22080112993717194
Final: 0.8608157634735107 0.22077421844005585
@@@@@ 2
---- 0 0.7797667384147644 1.0436959266662598 0.36898988485336304 0.05353676155209541
---- 10000 0.3571315109729767 0.7976251244544983 0.6441654562950134 0.2782297134399414
